In [7]:
# coding: utf-8
import uuid
import json

import os

import corenlp


def extract_entity_sentence(text):
    os.environ["CORENLP_HOME"] = r'stanford-corenlp-full-2018-10-05'
    new_dict = []
    # set up the client
    with corenlp.CoreNLPClient(annotators=['tokenize','ssplit','pos','ner'], timeout=300000, memory='16G') as client:
        # submit the request to the server
        ann = client.annotate(text)
        
        for sen in ann.sentence:
            sen_dict = {}
            ners = []
            tokens = []
            pos = []
            dels = []
            for i in sen.token:
                tokens.append(i.word)
                ners.append(i.ner)
                pos.append(i.pos)
            sen_dict = {"token":tokens, "stanford_ner":ners, "stanford_pos":pos}
            new_dict.append(sen_dict)
        print(new_dict)
        return new_dict

def transfer_to_multi_relation(new_dict):
    result = []
    for i in new_dict:
    
            location_start = 0
            location_end = 0
            entity_count = 0
            subj_start = []
            subj_end = []
            subj_type = []
            subjType = ''
            
            for j in i['stanford_ner']:
                if subjType != j:
                    if j != 'O':
                        if subjType != 'O' and subjType != '':
                            subj_end.append(location_end)
                        subj_start.append(location_start)
                        subj_type.append(j)
                        subjType = j
                        location_end = location_start
                        entity_count += 1
                    else:
                        subjType = j
                        if location_start != 0:
                            subj_end.append(location_end)
                else:
                    location_end += 1
                location_start += 1
            if len(subj_end) != len(subj_start):
                subj_end.append(len(i['token']) - 1)
                
            if entity_count >= 2:
                sub_count = 0
                while sub_count < len(subj_start):
                    obj_count = 0
                    while obj_count < len(subj_start):
                        if subj_type[sub_count] == 'PERSON' or  subj_type[sub_count] == 'ORGANIZATION':
                            if sub_count != obj_count:
                                dict_so = {}
                                dict_so['id'] = str(uuid.uuid1())
                                dict_so['relation'] = 'no_relation'
                                dict_so['token'] = i['token']
                                dict_so['subj_start'] = subj_start[sub_count]
                                dict_so['subj_end'] = subj_end[sub_count]
                                dict_so['obj_start'] = subj_start[obj_count]
                                dict_so['obj_end'] = subj_end[obj_count]
                                dict_so['subj_type'] = subj_type[sub_count]
                                dict_so['obj_type'] = subj_type[obj_count]
                                dict_so['stanford_ner'] = i['stanford_ner']
                                result.append(dict_so)
                            obj_count += 1
                        else:
                            break
                    sub_count += 1
#             elif entity_count == 2:
#                 dict_so = {}
#                 dict_so['id'] = str(uuid.uuid1())
#                 dict_so['relation'] = 'no_relation'
#                 dict_so['token'] = i['token']
#                 dict_so['subj_start'] = subj_start[0]
#                 dict_so['subj_end'] = subj_end[0]
#                 dict_so['obj_start'] = subj_start[1]
#                 dict_so['obj_end'] = subj_end[1]
#                 dict_so['subj_type'] = subj_type[0]
#                 dict_so['obj_type'] = subj_type[1]
#                 result.append(dict_so)
    return result

In [8]:
#text = "The Adventure Travel and Responsible Tourism Conference and Mart 2010 conference sponsored by Pacific Asia Travel Association -LRB- PATA -RRB- concluded on Wednesday . The National Congress of American Indians was founded in 1944 in response to assimilation policies being imposed on tribes by the federal government. Founded in 1951, PATA is a non-profit membership association dedicated to building responsible development of the Asia Pacific travel and tourism industry. The Securities and Exchange Commission scheduled a news conference Thursday in Washington, DC, to discuss the allegations against Mozilo , who founded Countrywide in 1969 and was its chief executive until Bank of America purchased it last year as its financial condition deteriorated. Panama poised to withdraw from Central American Parliament PARLACEN , founded in 1991 , is based in Guatemala City and has six member states ."
with open('tacred/test.json', 'r') as f:
    res = json.load(f)
#res = extract_entity_sentence(text)
result = transfer_to_multi_relation(res)
with open('tacred/test1014.json', 'w', encoding='utf-8') as f:
    json.dump(result, f)

In [ ]:
!python3 code/run_tacred_1.py --model output  --do_eval --eval_test  --data_dir tacred  --eval_batch_size 32   --learning_rate 2e-5   --max_seq_length 128   --output_dir output --fp16

10/31/2019 04:16:22 - INFO - pytorch_pretrained_bert.tokenization - loading vocabulary file output/vocab.txt


In [6]:
with open('output/predictions.json', 'r') as f:
     data = json.load(f)

for d in data:
    output = {}
    output['entities'] = d[0]
    output['relations'] = d[1]
    output['sentence'] = d[2]
    #print(output)
    print(json.dumps(output, sort_keys=True, indent=2))
    

import spacy
from spacy import displacy
from collections import Counter

# nlp = spacy.load('en_core_web_sm')

# displacy.render(nlp(text), jupyter=True, style='ent')

{
  "entities": [
    {
      "DATE": [
        "2010",
        "Wednesday"
      ],
      "ORGANIZATION": [
        "Pacific Asia Travel Association",
        "PATA"
      ]
    }
  ],
  "relations": [
    {
      "entity1": "Pacific Asia Travel Association",
      "entity2": "PATA",
      "relation": "org:alternate_names",
      "score": 0.99951171875
    },
    {
      "entity1": "PATA",
      "entity2": "Pacific Asia Travel Association",
      "relation": "org:alternate_names",
      "score": 0.99951171875
    }
  ],
  "sentence": [
    {
      "sentence": "The Adventure Travel and Responsible Tourism Conference and Mart 2010 conference sponsored by Pacific Asia Travel Association ( PATA ) concluded on Wednesday ."
    }
  ]
}
{
  "entities": [
    {
      "DATE": [
        "1944"
      ],
      "ORGANIZATION": [
        "National Congress of American Indians"
      ]
    }
  ],
  "relations": [
    {
      "entity1": "National Congress of American Indians",
      "entity2": "1944"

In [ ]:
import spacy
from spacy import displacy
from collections import Counter

nlp = spacy.load('en_core_web_sm')

displacy.render(nlp(text), jupyter=True, style='ent')

In [9]:
result[0]

{'id': 'e88aad4a-fb94-11e9-91bc-42010a8a0003',
 'obj_end': 10,
 'obj_start': 10,
 'obj_type': 'LOCATION',
 'relation': 'no_relation',
 'stanford_ner': ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'LOCATION',
  'O',
  'PERSON',
  'PERSON',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'DATE',
  'O',
  'O',
  'O',
  'O',
  'PERSON',
  'PERSON',
  'O',
  'O',
  'O',
  'O',
  'O',
  'ORGANIZATION',
  'ORGANIZATION',
  'ORGANIZATION',
  'ORGANIZATION',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'ORGANIZATION',
  'ORGANIZATION',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 'subj_end': 13,
 'subj_start': 12,
 'subj_type': 'PERSON',
 'token': ['He',
  'has',
  'served',
  'as',
  'a',
  'policy',
  'aide',
  'to',
  'the',
  'late',
  'U.S.',
  'Senator',
  'Alan',
  'Cranston',
  ',',
  'as',
  'National',
  'Issues',
  'Director',
  'for',
  'the',
  '2004',
  'presidential',
  'campaign',
  'of',
  'Congressman',
  'Dennis',
